In [21]:
import pandas as pd
import requests
import json
import time

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAOAnQQEAAAAA9VgDWu9HfjGR5uo42wwzTro9He0%3Dk0OBHn5m92l0SWrwBREvEmfWzfNAuLTW46rupq2OfgXwscwSyu"

#Expects a twitter handle and potentially a token, returns a JSON-like object with all the tweets from that handle
#over the last week
def search_twitter(handle, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    query = "from:" + handle
    tweet_fields = "tweet.fields=text,created_at"
    max_results = "max_results=100&end_time=2021-06-21T01:27:00.000Z&start_time=2021-06-19T03:00:02.000Z"
    url = "https://api.twitter.com/1.1/tweets/search/recent?query={}&{}&{}".format(
        query, tweet_fields, max_results
    )
    print(url)
    
    response = requests.request("GET", url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [22]:
#Reading in the data for Congresspeople
congdf = pd.read_csv("CongressData.csv")
congdf.head()

,Congressperson,State,Party Affiliation,House or Senate,District,Race,Gender,Birthdate,Background,Twitter Handle
0,Jerry Carl,AL,R,House,1,White,M,6/17/58,Political,RepJerryCarl
1,Barry Moore,AL,R,House,2,White,M,9/26/66,Political,RepBarryMoore
2,Mike Rogers,AL,R,House,3,White,M,7/16/58,Political,RepMikeRogersAL
3,Robert Aderholt,AL,R,House,4,White,M,7/22/65,Legal,Robert_Aderholt
4,Mo Brooks,AL,R,House,5,White,M,4/29/54,Political,RepMoBrooks


In [23]:
#Creating empty DataFrame with the columns we will need for each tweet

tweetsdf = pd.DataFrame({
    'Name':[],
    'Twitter Handle':[],
    'Time':[],
    'Content': []
})
tweetsdf

,Name,Twitter Handle,Time,Content


In [24]:
#Query limit per 15 minutes is 300, so this will start returning 429 (Too many requests) errors if you're using a free developer account for the API
#Just change the starting point to wherever it fails and then resume after waiting 15 minutes
for i in range(452, len(congdf['Congressperson'])):
    if congdf['Twitter Handle'][i] != "None?":
        try:
            json_response = search_twitter(congdf["Twitter Handle"][i])
            for tweet in json_response["data"]:
                tweetsdf.loc[len(tweetsdf['Name'])] = [congdf['Congressperson'][i]] + [congdf['Twitter Handle'][i]] + [tweet['created_at']] + [tweet['text']]
        except Exception as exc:
            print(exc)
            if(exc == 429):
                print("Too many requests, waiting...")
                time.sleep(900)
                try:
                    json_response = search_twitter(congdf["Twitter Handle"][i])
                    for tweet in json_response["data"]:
                        tweetsdf.loc[len(tweetsdf['Name'])] = [congdf['Congressperson'][i]] + [congdf['Twitter Handle'][i]] + [tweet['created_at']] + [tweet['text']]
                except Exception as exc:
                    print(exc)

https://api.twitter.com/1.1/tweets/search/recent?query=from:MikeCrapo&tweet.fields=text,created_at&max_results=100&end_time=2021-06-21T01:27:00.000Z&start_time=2021-06-19T03:00:02.000Z
(404, '<!DOCTYPE html>\n<html lang="en" class="dog">\n  <head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Twitter / ?</title>\n    <meta name="version" content="1">\n    <link href="https://abs.twimg.com/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon">\n    <link rel="stylesheet" href="https://abs.twimg.com/errors/fullscreen_errors-39d97faf8c7cf33e502c3c015176efd9.css">\n  </head>\n  <body dir="auto">\n    <div class="top">\n      <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 1519 200">\n        <defs/>\n        <path d="M708 103l9-5 13 1 2-2 5 1h8l1-1 42-5h7l7 3v1l12-2a9368 9368 0 0129-6l2-5 12 1 6-2 3 1 2 1v-2l13-2v1l4 2 7-1h1l17-3 1 2 4-1 18-7 3-1 4-1h9l13-1 1-4 3-

KeyboardInterrupt: 

In [31]:
tweetsdf.tail()

,Name,Twitter Handle,Time,Content
1547,Ron Johnson,SenRonJohnson,2021-06-19T18:44:20.000Z,Happy Juneteenth Day from Milwaukee— one of th...
1548,Ron Johnson,SenRonJohnson,2021-06-19T16:14:37.000Z,I would like to dedicate thIs first official J...
1549,Tammy Baldwin,SenatorBaldwin,2021-06-19T16:21:31.000Z,"Today, we observe #Juneteenth, a day to reflec..."
1550,John Barrasso,SenJohnBarrasso,2021-06-19T19:20:18.000Z,Natrona County Meals on Wheels celebrates 50 y...
1551,Cynthia Lummis,SenLummis,2021-06-20T17:35:22.000Z,Happy Father’s Day! I’m grateful for the years...


In [29]:
congdf.loc[congdf['Congressperson'] == "Mazie Hirono"]

,Congressperson,State,Party Affiliation,House or Senate,District,Race,Gender,Birthdate,Background,Twitter Handle
451,Mazie Hirono,HI,D,Senate,Senate,Asian,F,11/3/47,"Legal, Political",maziehirono


In [32]:
tweetsdf.to_csv("Tweets06192021extra.csv")